In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython
%load_ext autotime

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
import pandas as pd

import numpy as np
import scipy as sp

from GELMMnet import GELMMnet
from sklearn.preprocessing import scale
from GELMMnet.utility.inference import _rmse
from GELMMnet.utility.kernels import kinship
from sklearn.metrics import mean_squared_error,r2_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 947 ms


# Comparing to GELnet

In [2]:


# set seed:
np.random.seed(42)


geno_filename = 'genotypes.csv'
X = np.genfromtxt(geno_filename)
n_s,n_f = X.shape

# simulate phenotype
n_c = 5
idx = np.random.randint(0,n_f,n_c)
w = 1./n_c * np.ones((n_c,1))
ypheno = np.dot(X[:,idx],w)
ypheno = (ypheno-ypheno.mean())/ypheno.std()
pheno_filename = 'phenotypes.csv'
ypop = np.genfromtxt(pheno_filename)
ypop = np.reshape(ypop,(n_s,1))
y = 0.3*ypop + 0.5*ypheno + 0.2*np.random.randn(n_s,1)
y = (y-y.mean())/y.std()


#genotype distance matrix
P = np.identity(X.shape[1])

n_train = 150
n_test = n_s - n_train
n_reps = 100
f_subset = 0.5


# split into training and testing
train_idx = np.random.permutation(np.arange(n_s))
test_idx = train_idx[n_train:]
train_idx = train_idx[:n_train]


Xtrain = X[train_idx,:]
Xtest = X[test_idx,:]


ytrain = y[train_idx]
ytest = y[test_idx]

# calculate kernel
K = kinship(X)
K[train_idx][:,train_idx]

#g = GELMMnet(ytrain, Xtrain, K=K[train_idx][:,train_idx], intercept=False, standardize=False)

#g.kfoldFit(P, nfold=5, cpu=4, alpha_nof=10, ratio_nof=11, debug=True)
#ypred = g.predict(Xtest)
#print("GridSearch:", _mse(ytest, ypred))
#print("sklearn", r2_score(ytest, ypred))

g2 = GELMMnet(ytrain, Xtrain, K=K[train_idx][:,train_idx], intercept=True)
SUX = g2.SUX
SUy = g2.SUy

b,w=g2.fit(P, l1=.01, l2=0.0)
print('... number of Nonzero Weights: %d'%(w!=0).sum())
print(w)
ypred = g2.predict(Xtest)
print("MSE:", _rmse(ytest, ypred))
print("r2:", r2_score(ytest, ypred))



... number of Nonzero Weights: 68
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.         -0.02310406  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.00690445  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.00179769  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.  

# Compare LMM-lasso

In [33]:
"""
lmm_lasso.py

Author:		Barbara Rakitsch
Year:		2012
Group:		Machine Learning and Computational Biology Group (http://webdav.tuebingen.mpg.de/u/karsten/group/)
Institutes:	Max Planck Institute for Developmental Biology and Max Planck Institute for Intelligent Systems (72076 Tuebingen, Germany)

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

import scipy as SP
import scipy.linalg as LA
import scipy.optimize as OPT
import pdb
import matplotlib.pylab as PLT
import time
import numpy as NP


def stability_selection(X,K,y,mu,n_reps,f_subset,**kwargs):
    """
    run stability selection

    Input:
    X: SNP matrix: n_s x n_f
    y: phenotype:  n_s x 1
    K: kinship matrix: n_s x n_s
    mu: l1-penalty

    n_reps:   number of repetitions
    f_subset: fraction of datasets that is used for creating one bootstrap

    output:
    selection frequency for all SNPs: n_f x 1
    """
    time_start = time.time()
    [n_s,n_f] = X.shape
    n_subsample = SP.ceil(f_subset * n_s)
    freq = SP.zeros(n_f)
    
    for i in range(n_reps):
        print('Iteration %d'%i)
        idx = SP.random.permutation(n_s)[:n_subsample]
        res = train(X[idx],K[idx][:,idx],y[idx],mu,**kwargs)
        snp_idx = (res['weights']!=0).flatten()
        freq[snp_idx] += 1.
        
    freq /= n_reps
    time_end = time.time()
    time_diff = time_end - time_start
    print('... finished in %.2fs'%(time_diff))
    return freq

def train(X,K,y,mu,numintervals=100,ldeltamin=-5,ldeltamax=5,rho=1,alpha=1,debug=False):
    """
    train linear mixed model lasso

    Input:
    X: SNP matrix: n_s x n_f
    y: phenotype:  n_s x 1
    K: kinship matrix: n_s x n_s
    mu: l1-penalty parameter
    numintervals: number of intervals for delta linesearch
    ldeltamin: minimal delta value (log-space)
    ldeltamax: maximal delta value (log-space)
    rho: augmented Lagrangian parameter for Lasso solver
    alpha: over-relatation parameter (typically ranges between 1.0 and 1.8) for Lasso solver

    Output:
    results
    """
    print('train LMM-Lasso')
    print('...l1-penalty: %.2f'%mu)
    
    time_start = time.time()
    [n_s,n_f] = X.shape
    assert X.shape[0]==y.shape[0], 'dimensions do not match'
    assert K.shape[0]==K.shape[1], 'dimensions do not match'
    assert K.shape[0]==X.shape[0], 'dimensions do not match'
    if y.ndim==1:
        y = SP.reshape(y,(n_s,1))

    # train null model
    S,U,ldelta0,monitor_nm = train_nullmodel(y,K,numintervals,ldeltamin,ldeltamax,debug=debug)
    print("ldelta", ldelta0)
    # train lasso on residuals
    delta0 = SP.exp(ldelta0)
    Sdi = 1./(S+delta0)
    Sdi_sqrt = SP.sqrt(Sdi)
    SUX = SP.dot(U.T,X)
    SUX = SUX * SP.tile(Sdi_sqrt,(n_f,1)).T
    SUy = SP.dot(U.T,y)
    SUy = SUy * SP.reshape(Sdi_sqrt,(n_s,1))
    
    w,monitor_lasso = train_lasso(SUX,SUy,mu,rho,alpha,debug=debug)

    time_end = time.time()
    time_diff = time_end - time_start
    print('... finished in %.2fs'%(time_diff))

    res = {}
    res['ldelta0'] = ldelta0
    res['weights'] = w
    res['time'] = time_diff
    res['monitor_lasso'] = monitor_lasso
    res['monitor_nm'] = monitor_nm
    return res


def predict(y_t,X_t,X_v,K_tt,K_vt,ldelta,w):
    """
    predict the phenotype

    Input:
    y_t: phenotype: n_train x 1
    X_t: SNP matrix: n_train x n_f
    X_v: SNP matrix: n_val x n_f
    K_tt: kinship matrix: n_train x n_train
    K_vt: kinship matrix: n_val  x n_train
    ldelta: kernel parameter
    w: lasso weights: n_f x 1

    Output:
    y_v: predicted phenotype: n_val x 1
    """
    print('predict LMM-Lasso')
    
    assert y_t.shape[0]==X_t.shape[0], 'dimensions do not match'
    assert y_t.shape[0]==K_tt.shape[0], 'dimensions do not match'
    assert y_t.shape[0]==K_tt.shape[1], 'dimensions do not match'
    assert y_t.shape[0]==K_vt.shape[1], 'dimensions do not match'
    assert X_v.shape[0]==K_vt.shape[0], 'dimensions do not match'
    assert X_t.shape[1]==X_v.shape[1], 'dimensions do not match'
    assert X_t.shape[1]==w.shape[0], 'dimensions do not match'
    
    [n_train,n_f] = X_t.shape
    n_test = X_v.shape[0]
    
    if y_t.ndim==1:
        y_t = SP.reshape(y_t,(n_train,1))
    if w.ndim==1:
        w = SP.reshape(w,(n_f,1))
    
    delta = SP.exp(ldelta)
    idx = w.nonzero()[0]

    if idx.shape[0]==0:
        return SP.dot(K_vt,LA.solve(K_tt + delta*SP.eye(n_train),y_t))
        
    y_v = SP.dot(X_v[:,idx],w[idx]) + SP.dot(K_vt, LA.solve(K_tt + delta*SP.eye(n_train),y_t-SP.dot(X_t[:,idx],w[idx])))
    return y_v

"""
helper functions
"""

def train_lasso(X,y,mu,rho=1,alpha=1,max_iter=5000,abstol=1E-4,reltol=1E-2,zero_threshold=1E-3,debug=False):
    """
    train lasso via Alternating Direction Method of Multipliers:
    min_w  0.5*sum((y-Xw)**2) + mu*|z|
    
    Input:
    X: design matrix: n_s x n_f
    y: outcome:  n_s x 1
    mu: l1-penalty parameter
    rho: augmented Lagrangian parameter
    alpha: over-relatation parameter (typically ranges between 1.0 and 1.8)

    the implementation is a python version of Boyd's matlab implementation of ADMM-Lasso, which can be found at:
    http://www.stanford.edu/~boyd/papers/admm/lasso/lasso.html

    more information about ADMM can be found in the paper linked at:
    http://www.stanford.edu/~boyd/papers/distr_opt_stat_learning_admm.html

    In particular, you can use any other Lasso-Solver instead. For the experiments, reported in the paper,
    we used the l1-solver from the package scikits. We didn't apply it here to avoid third-party packages.
    """
    if debug:
        print('... train lasso')

    # init
    [n_s,n_f] = X.shape
    w = SP.zeros((n_f,1))
    z = SP.zeros((n_f,1))
    u = SP.zeros((n_f,1))

    monitor = {}
    monitor['objval'] = []
    monitor['r_norm'] = []
    monitor['s_norm'] = []
    monitor['eps_pri'] = []
    monitor['eps_dual'] = []

    # cache factorization
    U = factor(X,rho)

    # save a matrix-vector multiply
    Xy = SP.dot(X.T,y)

    if debug:
        print('i\tobj\t\tr_norm\t\ts_norm\t\teps_pri\t\teps_dual')

    for i in range(max_iter):
        # w-update
        q = Xy + rho*(z-u)
        w = q/rho - SP.dot(X.T,LA.cho_solve((U,False),SP.dot(X,q)))/rho**2

        # z-update with relaxation
        zold = z
        w_hat = alpha*w + (1-alpha)*zold
        z = soft_thresholding(w_hat+u, mu/rho)

        # u-update
        u = u + (w_hat - z)

        monitor['objval'].append(lasso_obj(X,y,w,mu,z))
        monitor['r_norm'].append(LA.norm(w-z))
        monitor['s_norm'].append(LA.norm(rho*(z-zold)))
        monitor['eps_pri'].append(SP.sqrt(n_f)*abstol + reltol*max(LA.norm(w),LA.norm(z)))
        monitor['eps_dual'].append(SP.sqrt(n_f)*abstol + reltol*LA.norm(rho*u))

        if debug:
            print('%3d\t%10.4f\t%10.4f\t%10.4f\t%10.4f\t%10.2f'%(i,monitor['objval'][i],monitor['r_norm'][i],
                                                                 monitor['s_norm'][i],monitor['eps_pri'][i],monitor['eps_dual'][i]))

        if monitor['r_norm'][i]<monitor['eps_pri'][i] and monitor['r_norm'][i]<monitor['eps_dual'][i]:
            break

    
    w[SP.absolute(w)<zero_threshold]=0
    return w,monitor


def nLLeval(ldelta,Uy,S):
    """
    evaluate the negative log likelihood of a random effects model:
    nLL = 1/2(n_s*log(2pi) + logdet(K) + 1/ss * y^T(K + deltaI)^{-1}y,
    where K = USU^T.

    Uy: transformed outcome: n_s x 1
    S:  eigenvectors of K: n_s
    ldelta: log-transformed ratio sigma_gg/sigma_ee
    """
    n_s = Uy.shape[0]
    delta = SP.exp(ldelta)
    
    # evaluate log determinant
    Sd = S+delta
    ldet = SP.sum(SP.log(Sd))

    # evaluate the variance    
    Sdi = 1.0/Sd
    Uy = Uy.flatten()
    ss = 1./n_s * (Uy*Uy*Sdi).sum()

    # evalue the negative log likelihood
    nLL=0.5*(n_s*SP.log(2.0*SP.pi)+ldet+n_s+n_s*SP.log(ss));

    return nLL


def train_nullmodel(y,K,numintervals=100,ldeltamin=-5,ldeltamax=5,debug=False):
    """
    train random effects model:
    min_{delta}  1/2(n_s*log(2pi) + logdet(K) + 1/ss * y^T(K + deltaI)^{-1}y,
    
    Input:
    X: SNP matrix: n_s x n_f
    y: phenotype:  n_s x 1
    K: kinship matrix: n_s x n_s
    mu: l1-penalty parameter
    numintervals: number of intervals for delta linesearch
    ldeltamin: minimal delta value (log-space)
    ldeltamax: maximal delta value (log-space)
    """
    if debug:
        print('... train null model')
        
    n_s = y.shape[0]

    # rotate data
    S,U = LA.eigh(K)
    Uy = SP.dot(U.T,y)

    # grid search
    nllgrid=SP.ones(numintervals+1)*SP.inf
    ldeltagrid=SP.arange(numintervals+1)/(numintervals*1.0)*(ldeltamax-ldeltamin)+ldeltamin
    nllmin=SP.inf
    for i in SP.arange(numintervals+1):
        nllgrid[i]=nLLeval(ldeltagrid[i],Uy,S);
        
    # find minimum
    nll_min = nllgrid.min()
    ldeltaopt_glob = ldeltagrid[nllgrid.argmin()]

    # more accurate search around the minimum of the grid search
    for i in SP.arange(numintervals-1)+1:
        if (nllgrid[i]<nllgrid[i-1] and nllgrid[i]<nllgrid[i+1]):
            ldeltaopt,nllopt,iter,funcalls = OPT.brent(nLLeval,(Uy,S),(ldeltagrid[i-1],ldeltagrid[i],ldeltagrid[i+1]),full_output=True);
            if nllopt<nllmin:
                nllmin=nllopt;
                ldeltaopt_glob=ldeltaopt;

    monitor = {}
    monitor['nllgrid'] = nllgrid
    monitor['ldeltagrid'] = ldeltagrid
    monitor['ldeltaopt'] = ldeltaopt_glob
    monitor['nllopt'] = nllmin
    
    return S,U,ldeltaopt_glob,monitor
 

def factor(X,rho):
    """
    computes cholesky factorization of the kernel K = 1/rho*XX^T + I

    Input:
    X design matrix: n_s x n_f (we assume n_s << n_f)
    rho: regularizaer

    Output:
    L  lower triangular matrix
    U  upper triangular matrix
    """
    n_s,n_f = X.shape
    K = 1/rho*SP.dot(X,X.T) + SP.eye(n_s)
    U = LA.cholesky(K)
    return U


def soft_thresholding(w,kappa):
    """
    Performs elementwise soft thresholding for each entry w_i of the vector w:
    s_i= argmin_{s_i}  rho*abs(s_i) + rho/2*(x_i-s_i) **2
    by using subdifferential calculus

    Input:
    w vector nx1
    kappa regularizer

    Output:
    s vector nx1
    """
    n_f = w.shape[0]
    zeros = SP.zeros((n_f,1))
    s = NP.max(SP.hstack((w-kappa,zeros)),axis=1) - NP.max(SP.hstack((-w-kappa,zeros)),axis=1)
    s = SP.reshape(s,(n_f,1))
    return s


def lasso_obj(X,y,w,mu,z):
    """
    evaluates lasso objective: 0.5*sum((y-Xw)**2) + mu*|z|

    Input:
    X: design matrix: n_s x n_f
    y: outcome:  n_s x 1
    mu: l1-penalty parameter
    w: weights: n_f x 1
    z: slack variables: n_fx1

    Output:
    obj
    """
    return 0.5*((SP.dot(X,w)-y)**2).sum() + mu*SP.absolute(z).sum()



    # train null model
    S,U,ldelta0,monitor_nm = train_nullmodel(y,K,numintervals,ldeltamin,ldeltamax,debug=debug)
    print("ldelta", ldelt0)
    # train lasso on residuals
    delta0 = SP.exp(ldelta0)
    Sdi = 1./(S+delta0)
    Sdi_sqrt = SP.sqrt(Sdi)
    SUX = SP.dot(U.T,X)
    SUX = SUX * SP.tile(Sdi_sqrt,(n_f,1)).T
    SUy = SP.dot(U.T,y)
    SUy = SUy * SP.reshape(Sdi_sqrt,(n_s,1))
    
    w,monitor_lasso = train_lasso(SUX,SUy,mu,rho,alpha,debug=debug)

    time_end = time.time()
    time_diff = time_end - time_start
    print('... finished in %.2fs'%(time_diff))

    res = {}
    res['ldelta0'] = ldelta0
    res['weights'] = w
    res['time'] = time_diff
    res['monitor_lasso'] = monitor_lasso
    res['monitor_nm'] = monitor_nm
    return res

def test():
    # data directory
    data_dir = './'
    
    # load genotypes
    geno_filename = os.path.join(data_dir,'genotypes.csv')
    X = SP.genfromtxt(geno_filename)
    [n_s,n_f] = X.shape

    # simulate phenotype
    SP.random.seed(1)
    n_c = 5
    idx = SP.random.randint(0,n_f,n_c)
    w = 1./n_c * SP.ones((n_c,1))
    ypheno = SP.dot(X[:,idx],w)
    ypheno = (ypheno-ypheno.mean())/ypheno.std()
    pheno_filename = os.path.join(data_dir,'phenotypes.csv')
    ypop = SP.genfromtxt(pheno_filename)
    ypop = SP.reshape(ypop,(n_s,1))
    y = 0.3*ypop + 0.5*ypheno + 0.2*SP.random.randn(n_s,1)
    y = (y-y.mean())/y.std()
    
    # init
    debug = False
    n_train = 150
    n_test = n_s - n_train
    n_reps = 100
    f_subset = 0.5
    mu = 10

    # split into training and testing
    train_idx = SP.random.permutation(SP.arange(n_s))
    test_idx = train_idx[n_train:]
    train_idx = train_idx[:n_train]

    # calculate kernel
    K = 1./n_f*SP.dot(X,X.T)
    
    # train
    res = train(X[train_idx],K[train_idx][:,train_idx],y[train_idx],mu,debug=debug)
    w = res['weights']
    print('... number of Nonzero Weights: %d'%(w!=0).sum())
    ldelta0 = res['ldelta0']
    
    # predict
    yhat = predict(y[train_idx],X[train_idx,:],X[test_idx,:],K[train_idx][:,train_idx],K[test_idx][:,train_idx],ldelta0,w)
    #print(yhat)
    w1=w[:,0]
    print("w",w1[w1 != 0])
    print("r2:", r2_score(y[test_idx], yhat))
    corr = 1./n_test * ((yhat-yhat.mean())*(y[test_idx]-y[test_idx].mean())).sum()/(yhat.std()*y[test_idx].std())
    print('... corr(Yhat,Ytrue): %.2f (in percent)'%(corr))
    print()
    
    import time
    print("Fit GELMMnet", )
    g = GELMMnet(y[train_idx],X[train_idx,:],K=K[train_idx][:,train_idx],intercept=False)
    start = time.time()
    g.kfoldFit(SP.identity(n_f),l1_nof=100, cpu=1, l2_nof=10, debug=True, max_iter = 100, eps = 1e-05)
    #g.fit(SP.identity(n_f),l1=0.06833534, l2=1e-04, max_iter = 1000, eps = 1e-08)
    stop = time.time()
    SUX = g.SUX
    SUy = g.SUy
    #g.w=w1
    #g.b = 0
    print("CV took:", stop-start)
    yhat2 = g.predict(X[test_idx,:])
    yhat2 = SP.reshape(yhat2, (len(yhat2), 1))
    #print("w",g.w[g.w != 0])
    print("b",g.b)
    #print(yhat)
    print('... number of Nonzero Weights: %d'%(g.w !=0).sum())
    corr = 1./n_test * ((yhat2-yhat2.mean())*(y[test_idx]-y[test_idx].mean())).sum()/(yhat2.std()*y[test_idx].std())
    print("r2:", r2_score(y[test_idx], yhat2))
    print('... corr(Yhat,Ytrue): %.2f (in percent)'%(corr))
    print()
    return g
g=test()
SUX = g.SUX
SUy = g.SUy

train LMM-Lasso
...l1-penalty: 10.00
ldelta -0.988297084887
... finished in 0.80s
... number of Nonzero Weights: 14
predict LMM-Lasso
w [ 0.29842933 -0.00451417  0.26317048  0.02745376  0.01940525  0.30468686
 -0.00977561  0.0036733   0.01539739  0.07248216  0.23095535 -0.01170971
 -0.02254671  0.27645177]
r2: 0.785183005721
... corr(Yhat,Ytrue): 0.89 (in percent)

Fit GELMMnet
Correcting for population structure
Max l1: 0.341676704748
w != 0: []
L2: [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 0]
L1: [ 0.          0.00341677  0.00683353  0.0102503   0.01366707  0.01708384
  0.0205006   0.02391737  0.02733414  0.0307509   0.03416767  0.03758444
  0.0410012   0.04441797  0.04783474  0.05125151  0.05466827  0.05808504
  0.06150181  0.06491857  0.06833534  0.07175211  0.07516888  0.07858564
  0.08200241  0.08541918  0.08883594  0.09225271  0.09566948  0.09908624
  0.10250301  0.10591978  0.10933655  0.11275331  0.11617008  0.11958685
  0.12300361  0.12642038  0.12983715  0.1

In [64]:
%%R -i y,X,SUX,SUy
library("gelnet")

#w=gelnet.cv(SUX, SUy[,1], 10, 10)
#w
nL2=10
v1 <- 1:(as.integer( nL2 / 2 ))
if( nL2 %% 2 == 0 )
      vL2 <- 10 ^ c(rev(1-v1), v1)
else
      vL2 <- 10 ^ c(rev(-v1), 0, v1)
vL2

nL1 = 10
L1s = 0.341676704748
seq( 0, L1s, length.out = nL1+1 )[1:nL1]

 [1] 0.00000000 0.03416767 0.06833534 0.10250301 0.13667068 0.17083835
 [7] 0.20500602 0.23917369 0.27334136 0.30750903


time: 64 ms


In [61]:
import numpy as np
nL2 = 10
n = max(1,int(nL2/ 2))
print(n)
[10**x for x in range(-n,n)]+[0]


5


[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 0]

time: 4.54 ms


# Comparing with selectiveInference

In [32]:
%%R -o x,y,out
library(glmnet)
library(selectiveInference)
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
y = x%*%beta + sigma*rnorm(n)
# first run glmnet
#lambdahat = cv.glmnet(x,y,standardize=FALSE)
#print(lambdahat)
#sigmahat = estimateSigma(x,y, standardize=FALSE, intercept=F)$sigmahat

gfit = glmnet(x,y,standardize=FALSE)
# extract coef for a given lambda; note the 1/n factor!
# (and we don't save the intercept term) fixedLassoInf 7 
lambda = .8
args = coef(gfit, s=lambda/n, exact=TRUE)
print(args)
# compute fixed lambda p-values and selection intervals
out = fixedLassoInf(x,y,args[-1],lambda,sigma=sigma)
out

11 x 1 sparse Matrix of class "dgCMatrix"
                      1
(Intercept)  0.05496141
V1           2.90117931
V2           2.09140871
V3           0.10536154
V4           0.07723765
V5          -0.17150755
V6          -0.13135784
V7           0.13166391
V8           0.02265503
V9          -0.09731043
V10          .         

Call:
fixedLassoInf(x = x, y = y, beta = args[-1], lambda = lambda, 
    sigma = sigma)

Standard deviation of noise (specified or estimated) sigma = 1.000

Testing results at lambda = 0.800, with alpha = 0.100

 Var   Coef Z-score P-value LowConfPt UpConfPt LowTailArea UpTailArea
   1  2.912  18.897   0.000     2.655    3.196       0.048      0.049
   2  2.123  13.803   0.000     1.856    2.377       0.049      0.049
   3  0.141   0.817   0.495    -0.728    0.404       0.049      0.049
   4  0.109   0.699   0.578    -0.854    0.337       0.050      0.049
   5 -0.206  -1.240   0.257    -0.486    0.310       0.048      0.049
   6 -0.145  -0.968   0.358    -0.395

time: 51.1 ms


In [52]:
from GELMMnet import GELMMnet
l1=.8/float(n)
P = np.identity(x.shape[1])
g = GELMMnet(y, x)
print(g.train(P,l1=l1,l2=0))

g.post_selection_analysis(alpha=0.1,compute_intervals=True)

(-9.325873406851315e-17, array([ 2.90089583,  2.09186778,  0.10637913,  0.07805434, -0.17215038,
       -0.13101207,  0.13188417,  0.02279718, -0.09728396,  0.        ]))
Active [0 1 2 3 4 5 6 7 8]


/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:305: RuntimeWarning: invalid value encountered in double_scalars
  def _truncatednorm_surv(z, mean, a, b, sd):
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:313: RuntimeWarning: overflow encountered in exp
  if mean == -np.inf:
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/GELMMnet.py:315: RuntimeWarning: overflow encountered in exp
  


,pval,coef0,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
0.0,0.0,2.911576,2.900896,1.063790e+18,2.736984e-16,inf,0.0,0.0,2.736984e-18
1.0,0.0,2.122611,2.091868,7.770508e+17,2.731624e-16,inf,0.0,0.0,2.731624e-18
2.0,0.0,0.141078,0.106379,4.596589e+16,3.069196e-16,inf,0.0,0.0,3.069196e-18
3.0,0.0,0.109437,0.078054,3.936711e+16,2.779917e-16,inf,0.0,0.0,2.779917e-18
4.0,0.0,-0.205650,-0.172150,-6.981093e+16,-inf,-2.945816e-16,0.0,0.0,2.945816e-18
5.0,0.0,-0.144647,-0.131012,-5.448261e+16,-inf,-2.654913e-16,0.0,0.0,2.654913e-18
6.0,0.0,0.141072,0.131884,5.116789e+16,2.757051e-16,inf,0.0,0.0,2.757051e-18
7.0,0.0,0.045156,0.022797,1.711204e+16,2.638865e-16,inf,0.0,0.0,2.638865e-18
8.0,0.0,-0.112833,-0.097284,-4.066726e+16,-inf,-2.774537e-16,0.0,0.0,2.774537e-18


time: 2.59 s


time: 939 ms
